# Huisprijzen voorspellen - Intro

In deze workshop gaan we aan de slag met wat data analyse. De data die we gebruiken zijn funda-data. Gegevens van huizen en de vraagprijs. We gaan vandaag de data verkennen. Morgen gaan we een voorspellend model trainen

Dit notebook:
- data verkenning met pandas


In totaal zijn er 10 vragen in dit notebook. Ga lekker aan de slag en stuur een documentje met de antwoorden op je vragen op naar maya.sappelli@han.nl

_____________

# Predicting house prices - Intro

In this workshop, we will work with some data analysis. The data we will use is funda-data. Data of houses and the asking price. Today we are going to explore the data. Tomorrow we will train a predictive model

This notebook:
- data exploration with pandas


There are a total of 10 questions in this notebook. Get started and send a document with the answers to your questions to maya.sappelli@han.nl

Translated with www.DeepL.com/Translator (free version)

# Data
Eerst ga je de dataset downloaden (zie volgende code block)


Deze dataset ga je exploreren. Je wilt ontdekken hoeveel data je hebt, wat de verdelingen zijn en wat voor relaties er in de dataset zitten. 

We beginnen met het beschrijven van de data
________
# Data
First you are going to download the dataset (see next code block)


You are going to explore this dataset. You want to discover how much data you have, what the distributions are and what kind of relationships are in the dataset. 

We start by describing the data

In [ ]:
#download de funda dataset hier: / download the funda dataset here:
!wget -q https://msappelli.github.io/teaching/nl_for_sale_all_anon.json 

In [2]:
# data inlezen en beschrijven / data reading and describing
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#The downloaded data is stored locally and we load it into a DataFrame using pandas
df = pd.read_json("nl_for_sale_all_anon.json")

#first we are going to look at how the data is put together and what attributes we all have about the houses in the dataset
df.info()

#also look at some elements of the dataset
df.head()

df.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8144 entries, 0 to 8143
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   crawled_in        8144 non-null   int64  
 1   vraagprijs        8144 non-null   object 
 2   vraagprijsm2      8144 non-null   float64
 3   bouwjaar          7750 non-null   object 
 4   woonoppervlakte   8144 non-null   int64  
 5   perceel           8144 non-null   object 
 6   kamers            8144 non-null   int64  
 7   slaapkamers       8144 non-null   int64  
 8   stad              8144 non-null   object 
 9   aangeboden_sinds  8144 non-null   object 
 10  item_type         2978 non-null   object 
 11  latitude          8144 non-null   float64
 12  longitude         8144 non-null   float64
dtypes: float64(3), int64(4), object(6)
memory usage: 890.8+ KB


Index(['crawled_in', 'vraagprijs', 'vraagprijsm2', 'bouwjaar',
       'woonoppervlakte', 'perceel', 'kamers', 'slaapkamers', 'stad',
       'aangeboden_sinds', 'item_type', 'latitude', 'longitude'],
      dtype='object')

In [38]:
# Translating the column names
translate_cols = {
    'crawled_in':'year_crawled', 
    'vraagprijs':'price', 
    'vraagprijsm2':'price_m2', 
    'bouwjaar':'year_construction',
    'woonoppervlakte':'area_living', 
    'perceel':'land_plot', 
    'kamers':'room_no', 
    'slaapkamers':'berooms_no', 
    'stad':'city',      
    'aangeboden_sinds':'offered_since', 
    'item_type':'item_type', 
    'latitude':'latitude', 
    'longitude':'longitude'
}
df2 = df.rename(translate_cols, axis = 'columns')
translate_cols['woonoppervlakte']

'area_living'

In [39]:
df2


,year_crawled,price,price_m2,year_construction,area_living,land_plot,room_no,berooms_no,city,offered_since,item_type,latitude,longitude
0,2022,325.000,3095.238095,1987,105,164,5,4,Arnhem,2 september 2022,"Eengezinswoning, eindwoning",51.9577,5.8930
1,2022,250.000,4032.258065,1954,62,,3,2,Arnhem,1 september 2022,None,51.9911,5.9366
2,2022,270.000,3417.721519,None,79,,2,1,Arnhem,6 weken,None,51.9814,5.9064
3,2022,269.500,2542.452830,1980,106,123,4,3,Arnhem,2 maanden,"Eengezinswoning, tussenwoning",51.9484,5.8577
4,2022,650.000,4744.525547,1934,137,262,5,4,Arnhem,8 weken,"Eengezinswoning, hoekwoning",51.9922,5.8823
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8139,2021,197.500,1899.038462,1867,104,313,4,3,Enschede,2 weken,"Eengezinswoning, tussenwoning",52.2201,6.9615
8140,2021,795.000,2760.416667,1923,288,652,11,5,Enschede,2 weken,"Villa, vrijstaande woning",52.2257,6.8956
8141,2021,325.000,2338.129496,1967,139,195,5,3,Enschede,2 weken,"Eengezinswoning, geschakelde woning",52.2361,6.8945
8142,2021,230.000,3333.333333,1953,69,165,5,3,Enschede,2 weken,"Eengezinswoning, tussenwoning",52.2125,6.8641


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8144 entries, 0 to 8143
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   crawled_in         8144 non-null   int64  
 1   price              8144 non-null   object 
 2   price_m2           8144 non-null   float64
 3   year_construction  7750 non-null   object 
 4   area_living        8144 non-null   int64  
 5   land_plot          8144 non-null   object 
 6   room_no            8144 non-null   int64  
 7   berooms_no         8144 non-null   int64  
 8   city               8144 non-null   object 
 9   offered_since      8144 non-null   object 
 10  item_type          2978 non-null   object 
 11  latitude           8144 non-null   float64
 12  longitude          8144 non-null   float64
dtypes: float64(3), int64(4), object(6)
memory usage: 890.8+ KB


In [42]:
df2['year_crawled'].value_counts()

2021    6183
2022    1961
Name: year_crawled, dtype: int64

## Vragen
1. Hoeveel missende perceelgroottes heb je?
2. Welke verschillende formats zie je in 'aangeboden sinds'
3. Welke item-types zijn er?


Voor het beantwoorden van deze vragen kun je de volgende functies gebruiken:
- `df['kolomnaam'].unique()` https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html
- `df['kolomnaam'].value_counts()` : https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html
_________
## Questions
1. How many missing plot sizes do you have?
2. What different formats do you see in 'offered since'
3. What item types are there?


To answer these questions, you can use the following functions:
- `df['column name'].unique()` https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html
- `df['column name'].value_counts()` : https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html

In [24]:
# code hier
plot_count = df2['land_plot'].value_counts()
plot_count

       5463
1        63
145      31
160      31
115      30
       ... 
765       1
466       1
529       1
5         1
721       1
Name: land_plot, Length: 515, dtype: int64

In [27]:
# Answer 1
plot_count['']

5463

In [29]:
# code hier
df2['offered_since'].unique()

# d month yyy, time difference in months or weeks

array(['2 september 2022', '1 september 2022', '6 weken', '2 maanden',
       '8 weken', '4 maanden', '5 weken', '2 weken', '5 maanden',
       '3 maanden', '6 september 2022', '13 september 2022',
       '12 september 2022', '14 september 2022', '6+ maanden', 'Vandaag',
       '3 weken', '4 weken', '5 september 2022', '7 september 2022',
       '8 september 2022', '7 weken', '9 september 2022',
       '10 september 2022', '3 september 2022', '4 september 2022',
       '11 september 2022', '2 oktober 2021', '8 oktober 2021',
       '9 oktober 2021', '30 september 2021', '7 oktober 2021',
       '28 september 2021', '29 september 2021', '6 oktober 2021',
       '5 oktober 2021', '27 september 2021', '4 oktober 2021',
       '1 oktober 2021', '3 oktober 2021'], dtype=object)

In [31]:
len(df2['offered_since'].unique())

40

In [33]:
df2['item_type'].unique()

array(['Eengezinswoning, eindwoning', None,
       'Eengezinswoning, tussenwoning', 'Eengezinswoning, hoekwoning',
       'Herenhuis, 2-onder-1-kapwoning',
       'Eengezinswoning, 2-onder-1-kapwoning',
       'Eengezinswoning, geschakelde woning',
       'Eengezinswoning, vrijstaande woning', 'Villa, vrijstaande woning',
       'Eengezinswoning, halfvrijstaande woning',
       'Bungalow, geschakelde woning', 'Herenhuis, tussenwoning',
       'Eengezinswoning, 2-onder-1-kapwoning (split-level woning)',
       'Woonboerderij, vrijstaande woning', 'Herenhuis, hoekwoning',
       'Eengezinswoning, geschakelde 2-onder-1-kapwoning',
       'Woonboerderij, halfvrijstaande woning',
       'Herenhuis, halfvrijstaande woning',
       'Eengezinswoning, tussenwoning (drive-in woning)',
       'Eengezinswoning, 2-onder-1-kapwoning (hofjeswoning)',
       'Eengezinswoning, tussenwoning (waterwoning)',
       'Eengezinswoning, hoekwoning (drive-in woning)',
       'Eengezinswoning, hoekwoning (split

In [34]:
# code hier
len(df2['item_type'].unique())

87

# Data Formats

Soms wordt de data niet goed ingelezen. Bijvoorbeeld getallen die als tekst worden geinterpreteerd in plaats van als getal.

Bekijk voor elke kolom of het type data klopt. Wanneer dit niet het geval is zet je de data om in het goede formaat.

_____________
# Data Formats

Sometimes data is not read in properly. For example, numbers interpreted as text instead of numbers.

For each column, check whether the data type is correct. If not, convert the data to the correct format.


## Vragen
4.  Welke kolommen hadden niet het juiste formaat? En welke omzetting heb je gedaan?

Voor het beantwoorden van deze vraag kun je gebruik maken van de volgende functies:
- `df.info()` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html
- `df["kolomnaam"] = pd.to_numeric(df["kolomnaam"])` https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html
- `df["kolomnaam"] = df["kolomnaam"].str.replace("x","y"))` om in een kolom bepaalde tekstuele vervangingen te doen. Hierin kun je ook reguliere expressies gebruiken zoals `\w+\s+` https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html
______________
## Questions
4.  Which columns did not have the correct format? And what conversion did you do?

To answer this question, you can use the following functions:
- `df.info()` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html
- `df["column name"] = pd.to_numeric(df["column name"])` https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html
- `df["column name"] = df["column name"].str.replace("x", "y"))` to do certain textual substitutions in a column. In this, you can also use regular expressions such as `\w+\s+` https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html

In [35]:
# code hier
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8144 entries, 0 to 8143
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   crawled_in         8144 non-null   int64  
 1   price              8144 non-null   object 
 2   price_m2           8144 non-null   float64
 3   year_construction  7750 non-null   object 
 4   area_living        8144 non-null   int64  
 5   land_plot          8144 non-null   object 
 6   room_no            8144 non-null   int64  
 7   berooms_no         8144 non-null   int64  
 8   city               8144 non-null   object 
 9   offered_since      8144 non-null   object 
 10  item_type          2978 non-null   object 
 11  latitude           8144 non-null   float64
 12  longitude          8144 non-null   float64
dtypes: float64(3), int64(4), object(6)
memory usage: 890.8+ KB


In [37]:
df2[['crawled_in', 'offered_since']]

,crawled_in,offered_since
0,2022,2 september 2022
1,2022,1 september 2022
2,2022,6 weken
3,2022,2 maanden
4,2022,8 weken
...,...,...
8139,2021,2 weken
8140,2021,2 weken
8141,2021,2 weken
8142,2021,2 weken


Er zitten ook nog wat datapunten in de set waarvan de vraagprijs niet is ingevuld. Verwijder deze items, let op dat je niet teveel verwijderd/

Je kunt hiervoor `df.dropna()` gebruiken met de optie subset https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html

___________________________

There are also some data items in the set whose asking price is not filled in. Delete these items, being careful not to delete too many/

You can use `df.dropna()` for this with the subset option https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html

In [ ]:
# code hier

In [ ]:
# Bekijk hoe je data er nu uit ziet
df.head()

# Data Visualiseren

Vaak is data beter te begrijpen aan de hand van plaatjes. Daarom gaan we nu de verschillende kolommen visualiseren










## Vragen:
5. maak een histogram van bouwjaar
6. maak een histogram van het aantal kamers
7. *(optioneel) Probeer te spelen met het aantal groepen in het bouwjaar om de visualisatiee beter te maken, bijvoorbeeld bouwjaar 1950-1960*


gebruik `df['kolomnaam'].hist()` https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.hist.html - maak ook gebruik van de bins optie

In [ ]:
# code hier

Ook kunnen we kijken naar correlaties in de data

## Vragen
8. Maak een correlatiegrafiek tussen woonoppervlakte en vraagprijs
9. Zijn er nog andere combinaties van kolommen waarin een correlatie te zien is?

gebruik `df.plot.scatter()` https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.DataFrame.plot.scatter.html

In [ ]:
# code hier

In [ ]:
# je kunt ook nog experimenteren met
# df.boxplot() voor boxplots
# sns.heatmap(df.corr())

# Visualiseren van Locaties

In [ ]:
!pip install geopandas

Nu gaan we de huizen plotten op een kaart van Nederland met behulp van geopandas.

In [ ]:
import geopandas as gpd

#Eerst laden we een kaart van Nederland in
nl = gpd.read_file("https://stacks.stanford.edu/file/druid:st293bj4601/data.zip")
nlmap = nl.plot()

#Nu plotten we de huizen op basis van hun coordinaten op de kaart
houses = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df['longitude'], df['latitude']))
houses.plot(ax=nlmap, color='red', markersize=0.5)



## Vragen

10. Wat valt op aan de locaties van de huizen? 
11. Morgen gaan we verder met het voorspellen van huizenprijzen. Schrijf op welke informatie je denkt te kunnen gebruiken voor het voorspellen van de verkoopprijzen van huizen